In [ ]:
from prody import *
import numpy as np
from os.path import basename
import fnmatch
import os
import math

## Let's mutate the PDB structure


### First parse the pdb file and make a list of desired mutations


In [ ]:
pdb_name = "1fcc.pdb"
mutation_list = ["ALA","ARG","ASN","ASP","CYS","GLU","GLN","GLY","HIS","ILE",
                 "LEU","LYS","MET","PHE","PRO","SER","THR","TRP","TYR","VAL"]

### Here, we only mutate 'C chain' which is our primary protein and save mutated pdb files


In [ ]:
structure = parsePDB(pdb_name)
protein = structure.select("chain C")
partner = structure.select("chain A").copy()
protein = protein.copy()

res_nums = protein.getResnums()
res_nums = sorted(set(res_nums))
res_nums = np.array(res_nums)

for i in res_nums:

    protein_wh_res_sd = protein.select("protein and not (resnum %s and sidechain)" % str(i))
    protein_wh_res_sd = protein_wh_res_sd.copy()
    res_id_mut_checker = protein_wh_res_sd.select("resnum %s" % str(i))
    res_id_mut_checker = res_id_mut_checker.copy()
    for mutation in mutation_list:
        if res_id_mut_checker.select("calpha").getResnames() == mutation:
            continue
        res_id_mut_slc = protein_wh_res_sd.select("resnum %s" % str(i))
        res_id_mut_slc = res_id_mut_slc.setResnames(mutation)
    
        mutated_protein_structure = protein_wh_res_sd.copy()
        writePDB("protein_res{}_{}_mutated.pdb".format(str(i), mutation), mutated_protein_structure + partner)

### After generating the mutated pdb files, by using VMD we will solvate (put in a water box) and ionize (add salt) to the protein structure


In [ ]:
!vmd -dispdev text -e autopsf_batch.tcl
#if you use Mac and add vmd to PATH by giving it an alias
#!startup.command -dispdev text -e autopsf_batch.tcl
#alternatively, you can run 'vmd -dispdev text -e autopsf_batch.tcl' in your working directory, by using terminal

### To give multiple minimizations by using NAMD, we will generate configuration files for each mutation. Each minimization process will be 10,000 steps-long


In [ ]:
def config_gen(pdb_name,min_step_count):
    file_name_wh_ex = str(os.path.splitext(pdb_name)[0])
    structure = parsePDB(str(pdb_name))
    f = open(str(file_name_wh_ex)+"_config.conf", 'w')
    f.write("\n")
    f.write("%-10s\t\t\t%-10s\n" % ("structure",str(file_name_wh_ex)+".psf"))
    f.write("%-10s\t\t\t%-10s\n" % ("coordinates",str(file_name_wh_ex)+".pdb"))
    f.write("\n")
    f.write("\n")
    f.write("\n")
    f.write("set temperature    310\n")
    f.write("set outputname     %s\n" % (str(file_name_wh_ex)))
    f.write("\n")
    f.write("\n")
    f.write("firsttimestep      0\n")
    f.write("#############################################################\n")
    f.write("## SIMULATION PARAMETERS                                   ##\n")
    f.write("#############################################################\n")
    f.write("# Input\n")
    f.write("paraTypeCharmm      on\n")
    f.write("parameters          par_all36_carb.prm\n")
    f.write("parameters          par_all36_cgenff.prm\n")
    f.write("parameters          par_all36_lipid.prm\n")
    f.write("parameters          par_all36_na.prm\n")
    f.write("parameters          par_all36_prot.prm\n")
    f.write("parameters          toppar_water_ions.str\n")
    f.write("temperature         $temperature\n")
    f.write("\n")
    f.write("\n")
    f.write("# Force-Field Parameters\n")
    f.write("""
exclude             scaled1-4
1-4scaling          1.0
cutoff              12.
switching           on
switchdist          10.
pairlistdist        13.5
""")
    f.write("\n")
    f.write("\n")
    f.write("""
# Integrator Parameters
numsteps            1
timestep            2.0  ;# 2fs/step
rigidBonds          all  ;# needed for 2fs steps
nonbondedFreq       1
fullElectFrequency  2
stepspercycle       10
""")
    f.write("\n")
    f.write("\n")
    f.write("""
# Constant Temperature Control
langevin            on  ;# do langevin dynamics
langevinDamping     5     ;# damping coefficient (gamma) of 5/ps
langevinTemp        $temperature
langevinHydrogen    off    ;# don't couple langevin bath to hydrogens
""")
    f.write("\n")
    f.write("\n")

    ###################
    pdb_atom_coords = structure.getCoords()
    xmax = np.max(pdb_atom_coords[:, 0])
    ymax = np.max(pdb_atom_coords[:, 1])
    zmax = np.max(pdb_atom_coords[:, 2])
    xmin = np.min(pdb_atom_coords[:, 0])
    ymin = np.min(pdb_atom_coords[:, 1])
    zmin = np.min(pdb_atom_coords[:, 2])
    cb_vec1 = abs(xmax-xmin)+0.1
    cb_vec2 = abs(ymax-ymin)+0.1
    cb_vec3 = abs(zmax-zmin)+0.1
    cb_vec1_f = "{0:.1f}".format(cb_vec1)
    cb_vec2_f = "{0:.1f}".format(cb_vec2)
    cb_vec3_f = "{0:.1f}".format(cb_vec3)
    co_x = (xmax+xmin)/2
    co_y = (ymax+ymin)/2
    co_z = (zmax+zmin)/2
    co_x_f = "{0:.1f}".format(co_x)
    co_y_f = "{0:.1f}".format(co_y)
    co_z_f = "{0:.1f}".format(co_z)
    ###################

    f.write("# Periodic Boundary Conditions\n")
    f.write("cellBasisVector1     %s      0.0    0.0\n" % cb_vec1_f)
    f.write("cellBasisVector2     0.0       %s   0.0\n" % cb_vec2_f)
    f.write("cellBasisVector3     0.0       0.0    %s\n"% cb_vec3_f)
    f.write("cellOrigin           %s     %s     %s\n" % (co_x_f, co_y_f, co_z_f))
    f.write("\n")
    f.write("\n")
    f.write("wrapall              on\n")
    f.write("\n")
    f.write("\n")
    f.write("""
# PME (for full-system periodic electrostatics)
PME                 yes
pmeGridSpacing      1.0
#PMEGridSizeX        90
#PMEGridSizeY       108
#PMEGridSizeZ       108


# Constant Pressure Control (variable volume)
useGroupPressure      yes ;# needed for rigidBonds
useFlexibleCell       no
useConstantArea       no

langevinPiston        yes
langevinPistonTarget  1.01325 ;#  in bar -> 1 atm
langevinPistonPeriod  100.
langevinPistonDecay   50.
langevinPistonTemp    $temperature


# Output
outputName          $outputname
dcdfreq             1000
DCDfile             $outputname.dcd
outputEnergies      1000
outputPressure      1000


#############################################################
## EXTRA PARAMETERS                                        ##
#############################################################


#############################################################
## EXECUTION SCRIPT                                        ##
#########################################################
""")
    f.write("\n")
    f.write("\n")
    f.write("minimize %s" % (str(min_step_count)))
    f.write("\n")
    f.close()

    return

global step_size
step_size = 10000

for file in os.listdir('.'):
    if fnmatch.fnmatch(file, '*ionized.pdb'):
        pdb = file
        config_gen(str(pdb),str(step_size))


### We will compile multiple configuration files into one bash job file to give batch minimization runs


In [ ]:
f = open("config_joblist", 'w')

f.write("#!/bin/bash\n")
f.write("echo config jobs started\n")
file_names_sorted = []
for file in os.listdir('.'):
    if fnmatch.fnmatch(file, '*_config.conf'):
        file_name_wh_ex = str(os.path.splitext(file)[0])
        file_names_sorted.append(file_name_wh_ex)
        file_names_sorted = sorted(file_names_sorted, key=str.lower)

range_list = len(file_names_sorted)

for i in range(range_list):
    conffile = str(file_names_sorted[i] +".conf")
    logfile = str(file_names_sorted[i] +".log")
    f.write("namd2 +p64 %s > %s\n" % (str(conffile), str(logfile)))
    f.write("echo job for %s is finished\n" % str(conffile))

f.write(" echo all jobs are finished")
f.close()


### Finally, we will extract minimized mutated conformation from the trajectory files


In [ ]:
def lf_extract(pdb_name):
    file_name_wh_ex = str(os.path.splitext(pdb_name)[0])
    structure = parsePDB(str(pdb_name))
    traj = Trajectory(str(file_name_wh_ex)+".dcd")
    traj.link(structure)
    last_frame = traj[-1]
    last_frame = last_frame.getAtoms()
    last_frame = last_frame.select("protein")
    writePDB(str(file_name_wh_ex)+"_lf_full.pdb", last_frame)
    return

for file in os.listdir('.'):
    if fnmatch.fnmatch(file, 'protein*mutated_autopsf_wb_ionized.pdb'):
        pdb = file
        lf_extract(str(pdb))